In [1]:
import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPRO
from dspy.evaluate import Evaluate
from dspy.teleprompt.ensemble import Ensemble
from datasets import load_dataset
import random
import re
import os

# save all our hard work in the prompts directory
os.makedirs('prompts', exist_ok=True)

random.seed(0)

COMPILE_FS = False
COMPILE_MIPRO = False
COMPILE_COT = False

/home/toolkit/.conda/envs/dspy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = dspy.HFClientVLLM(model="meta-llama/Meta-Llama-3-8B-Instruct",
                       port=8080,
                       url="http://localhost")
dspy.settings.configure(lm=lm)
NUM_THREADS = 8

# Dataset

* Create a dataset using the dspy.Example class
* We will use the Ultra Feedback dataset
    * 1 instruction
    * 4 possible completions
    * all of them are rated by gpt4
* Our goal is to find a good prompt to get the best RM out of Llama3
* We will create a train, valid, and test dataset
* We will evaluate our model using exact match

In [3]:
dataset = load_dataset("openbmb/UltraFeedback")
all_data = []
for input_ in dataset["train"]:
    completions = sorted(
        input_["completions"], key=lambda x: x["overall_score"], reverse=True
    )
    # only take the top and bottom completions
    for i, chosen_completion in enumerate(completions[:1]):
        for rejected_completion in completions[-1:]:
            if (
                chosen_completion["overall_score"]
                == rejected_completion["overall_score"]
            ):
                continue
            if random.random() < 0.5:
                text1 = chosen_completion["response"]
                text2 = rejected_completion["response"]
                preference = "1"
            else:
                text1 = rejected_completion["response"]
                text2 = chosen_completion["response"]
                preference = "2"
                
            if text1 == text2:
                continue
            # llama 3 has a smart context window
            if len(text1) > 1524 or len(text2) > 1524 or len(input_["instruction"]) > 1524:
                continue
            
            all_data.append(
                dspy.Example(
                    **{
                        "instruction": input_["instruction"],
                        "text_1": text1,
                        "text_2": text2,
                        "preferred_text": preference,
                    }
                ).with_inputs("instruction", "text_1", "text_2")
            )
    if len(all_data) > 2000:
        break

random.shuffle(all_data)

valid = all_data[:100]
test = all_data[100:600]
train = all_data[600:]

/home/toolkit/.conda/envs/dspy/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
def extract_pref(example, pred):
    pref_pred = re.search(r"\d+", pred["preferred_text"])
    if pref_pred:
        pref_pred = pref_pred.group()
    else:
        pref_pred = None
    return pref_pred


def em_metric(example, pred, trace=None, frac=1.0, verbose=False):
    pref_pred = extract_pref(example, pred)
    if verbose:
        print(f"Example: {example['preferred_text']}")
        print(f"Prediction: {pref_pred}")
    score = example["preferred_text"] == pref_pred
    if score is None:
        return False
    return score

eval_fn = Evaluate(devset=test, metric=em_metric, num_threads=NUM_THREADS, display_progress=True)

In [5]:
class Preference(dspy.Signature):
    instruction = dspy.InputField()
    text_1 = dspy.InputField()
    text_2 = dspy.InputField()
    preferred_text = dspy.OutputField(desc="Only return the preferred text (1 or 2) as an int", prefix="preferred_text:")

class PrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.Predict(
            Preference, max_tokens=3, temperature=0.1
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
                        instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [6]:
pred = PrefPredict()(**train[0])
print(pred)
em_metric(train[0], pred, verbose=True)

Prediction(
    preferred_text='1'
)
Example: 1
Prediction: 1


True

In [7]:
lm.inspect_history(n=1)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Follow the following format.

Instruction: ${instruction}

Text 1: ${text_1}

Text 2: ${text_2}

preferred_text: Only return the preferred text (1 or 2) as an int

---

Instruction: For a given set of 10 food-related words, provide the frequency of their usage in a corpus of reviews and the average number of syllables for each word. Furthermore, can you identify the top 3 food-related words with the highest frequency of usage and calculate the standard deviation of their syllable counts across the corpus?

Text 1: Here are the frequencies of the ten food-related words in a corpus of restaurant reviews, along with the average number of syllables: - Pasta: 188 occurrences, 1.5 syllables - Salad: 157 occurrences, 1.7 syllables - Bread: 128 occurrences, 1.5 syllables - Chicken: 154 occurrences, 2.1 syllables - Rice: 115 occurrences, 1.5 syllables - Fish: 121 occurrences, 2.0 syllables - Meat:

In [8]:
eval_fn(PrefPredict())

Average Metric: 3 / 4  (75.0):   1%|          | 3/500 [00:00<00:01, 477.06it/s]  

Average Metric: 324 / 500  (64.8): 100%|██████████| 500/500 [00:05<00:00, 86.71it/s] 

Average Metric: 324 / 500  (64.8%)


64.8

# Bootstrapping Few Shot Example with Random Search

* Need a train and a valid set, a metric, and a LLM.
* Does not require demonstration, e.g., no demo CoT will be required.
* The LLM will create good traces that satisfy the metric.
* We will use random search to find the best prompt over generated traces and input/output pairs.

In [9]:
boot_fs = BootstrapFewShotWithRandomSearch(metric=em_metric, max_bootstrapped_demos=4, max_labeled_demos=4,
                                           num_threads=NUM_THREADS, max_rounds=1, num_candidate_programs=50)

if COMPILE_FS:
    preference_model = boot_fs.compile(PrefPredict(), trainset=train, valset=valid)
    ensemble_preference_model = [prog for *_, prog in preference_model.candidate_programs[:3]]
    for idx, prog in enumerate([x[-1] for x in ensemble_preference_model]):
        prog.save(f'prompts/preference_model_{idx}.json')
else:
    ensemble_preference_model = []
    for idx in range(3):
        prog = PrefPredict()
        prog.load(f'prompts/preference_model_{idx}.json')
        ensemble_preference_model.append(prog)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 50 candidate sets.


Average Metric: 68 / 100  (68.0): 100%|██████████| 100/100 [00:01<00:00, 84.74it/s]


Average Metric: 68 / 100  (68.0%)
Score: 68.0 for set: [0]
New best score: 68.0 for seed -3
Scores so far: [68.0]
Best score: 68.0


Average Metric: 74 / 100  (74.0): 100%|██████████| 100/100 [00:06<00:00, 15.81it/s]


Average Metric: 74 / 100  (74.0%)
Score: 74.0 for set: [4]
New best score: 74.0 for seed -2
Scores so far: [68.0, 74.0]
Best score: 74.0


...

Average Metric: 70 / 100  (70.0%)
Score: 70.0 for set: [4]
Scores so far: [68.0, 74.0, 63.0, 73.0, 68.0, 74.0, 69.0, 82.0, 68.0, 69.0, 63.0, 69.0, 72.0, 70.0, 78.0, 69.0, 70.0, 55.0, 63.0, 70.0, 71.0, 70.0, 74.0, 65.0, 61.0, 50.0, 68.0, 72.0, 52.0, 70.0, 73.0, 72.0, 71.0, 71.0, 67.0, 78.0, 70.0, 64.0, 73.0, 72.0, 68.0, 50.0, 66.0, 74.0, 65.0, 70.0, 64.0, 73.0, 74.0, 68.0, 73.0, 70.0]
Best score: 82.0
Average of max per entry across top 1 scores: 0.82
Average of max per entry across top 2 scores: 0.96
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0


  0%|          | 1/1401 [00:00<01:42, 13.65it/s]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 53 / 100  (53.0): 100%|██████████| 100/100 [00:04<00:00, 21.29it/s]

Average Metric: 53 / 100  (53.0%)
Score: 53.0 for set: [4]
Scores so far: [68.0, 74.0, 63.0, 73.0, 68.0, 74.0, 69.0, 82.0, 68.0, 69.0, 63.0, 69.0, 72.0, 70.0, 78.0, 69.0, 70.0, 55.0, 63.0, 70.0, 71.0, 70.0, 74.0, 65.0, 61.0, 50.0, 68.0, 72.0, 52.0, 70.0, 73.0, 72.0, 71.0, 71.0, 67.0, 78.0, 70.0, 64.0, 73.0, 72.0, 68.0, 50.0, 66.0, 74.0, 65.0, 70.0, 64.0, 73.0, 74.0, 68.0, 73.0, 70.0, 53.0]
Best score: 82.0
Average of max per entry across top 1 scores: 0.82
Average of max per entry across top 2 scores: 0.96
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
53 candidate programs found.


In [11]:
eval_fn(ensemble_preference_model[0])

  0%|          | 0/500 [00:00<?, ?it/s]

Average Metric: 342 / 500  (68.4): 100%|██████████| 500/500 [00:25<00:00, 19.47it/s]

Average Metric: 342 / 500  (68.4%)


68.4

In [18]:
lm.inspect_history(n=1)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Instruction: Recognize whether the following phrase is in passive voice and identify the subject and object of the sentence. Additionally, determine the tense of the verb and indicate if the action was completed or ongoing at a specific point in time. #The Given Prompt#: Create a function that takes a list of integers and returns the largest product that can be made by multiplying any three integers. ``` def largest_product(lst): pass ``` #Rewritten Prompt#: Create a function that takes a list of integers and returns the largest product that can be made by multiplying any three distinct integers. However, if any of the integers are negative, the function should only consider the two smallest negative integers and the largest positive integer in the list for the calculation of the product. If there are no negative integers, the function should consider the three largest integers in the lis

# Using an ensemble

In [12]:
ensemble_optimizer = Ensemble(reduce_fn=dspy.majority)
ensemble_preference_model_fn = ensemble_optimizer.compile(ensemble_preference_model)
eval_fn(ensemble_preference_model_fn)

Average Metric: 345 / 500  (69.0): 100%|██████████| 500/500 [00:57<00:00,  8.68it/s]

Average Metric: 345 / 500  (69.0%)


69.0

# Adding Chain of Thought

In [13]:
class CoTPrefPredict(dspy.Module):
    def __init__(self):
        super().__init__()
        self.preference = dspy.ChainOfThought(
            Preference
        ) 

    def forward(self, instruction, text_1, text_2, *args, **kwargs):
        preferred = self.preference(
            instruction=instruction,
            text_1=text_1,
            text_2=text_2,
        )
        return preferred

In [14]:
if COMPILE_COT:
    cot_preference_model = boot_fs.compile(CoTPrefPredict(), trainset=train, valset=valid)
    ensemble_cot_preference_model = []
    for idx, prog in enumerate([x[-1] for x in cot_preference_model.candidate_programs[:3]]):
        prog.save(f'prompts/cot_preference_model_{idx}.json')
        ensemble_cot_preference_model.append(prog)
else:
    ensemble_cot_preference_model = []
    for idx in range(3):
        prog = CoTPrefPredict()
        prog.load(f'prompts/cot_preference_model_{idx}.json')
        ensemble_cot_preference_model.append(prog)
    

Average Metric: 1 / 1  (100.0):   0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 63 / 100  (63.0): 100%|██████████| 100/100 [00:18<00:00,  5.37it/s]


Average Metric: 63 / 100  (63.0%)
Score: 63.0 for set: [0]
New best score: 63.0 for seed -3
Scores so far: [63.0]
Best score: 63.0


Average Metric: 66 / 100  (66.0): 100%|██████████| 100/100 [00:30<00:00,  3.23it/s]


Average Metric: 66 / 100  (66.0%)
Score: 66.0 for set: [4]
New best score: 66.0 for seed -2
Scores so far: [63.0, 66.0]
Best score: 66.0


...

Average Metric: 71 / 100  (71.0%)
Score: 71.0 for set: [4]
Scores so far: [63.0, 66.0, 66.0, 71.0, 69.0, 71.0, 75.0, 77.0, 68.0, 69.0, 51.0, 71.0, 72.0, 72.0, 66.0, 72.0, 70.0, 55.0, 71.0, 73.0, 73.0, 70.0, 76.0, 76.0, 68.0, 49.0, 65.0, 66.0, 59.0, 71.0, 69.0, 71.0, 72.0, 63.0, 61.0, 76.0, 63.0, 65.0, 71.0, 74.0, 70.0, 65.0, 71.0, 66.0, 71.0, 66.0, 65.0, 67.0, 67.0, 63.0, 69.0, 71.0]
Best score: 77.0
Average of max per entry across top 1 scores: 0.77
Average of max per entry across top 2 scores: 0.97
Average of max per entry across top 3 scores: 0.98
Average of max per entry across top 5 scores: 0.99
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0


  0%|          | 1/1401 [00:01<45:43,  1.96s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 56 / 100  (56.0): 100%|██████████| 100/100 [00:31<00:00,  3.15it/s]

Average Metric: 56 / 100  (56.0%)
Score: 56.0 for set: [4]
Scores so far: [63.0, 66.0, 66.0, 71.0, 69.0, 71.0, 75.0, 77.0, 68.0, 69.0, 51.0, 71.0, 72.0, 72.0, 66.0, 72.0, 70.0, 55.0, 71.0, 73.0, 73.0, 70.0, 76.0, 76.0, 68.0, 49.0, 65.0, 66.0, 59.0, 71.0, 69.0, 71.0, 72.0, 63.0, 61.0, 76.0, 63.0, 65.0, 71.0, 74.0, 70.0, 65.0, 71.0, 66.0, 71.0, 66.0, 65.0, 67.0, 67.0, 63.0, 69.0, 71.0, 56.0]
Best score: 77.0
Average of max per entry across top 1 scores: 0.77
Average of max per entry across top 2 scores: 0.97
Average of max per entry across top 3 scores: 0.98
Average of max per entry across top 5 scores: 0.99
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
53 candidate programs found.


In [17]:
eval_fn(ensemble_cot_preference_model[0])

Average Metric: 352 / 500  (70.4): 100%|██████████| 500/500 [02:17<00:00,  3.63it/s]

Average Metric: 352 / 500  (70.4%)


70.4

In [19]:
lm.inspect_history(n)





Given the fields `instruction`, `text_1`, `text_2`, produce the fields `preferred_text`.

---

Instruction: Recognize whether the following phrase is in passive voice and identify the subject and object of the sentence. Additionally, determine the tense of the verb and indicate if the action was completed or ongoing at a specific point in time. #The Given Prompt#: Create a function that takes a list of integers and returns the largest product that can be made by multiplying any three integers. ``` def largest_product(lst): pass ``` #Rewritten Prompt#: Create a function that takes a list of integers and returns the largest product that can be made by multiplying any three distinct integers. However, if any of the integers are negative, the function should only consider the two smallest negative integers and the largest positive integer in the list for the calculation of the product. If there are no negative integers, the function should consider the three largest integers in the lis

In [22]:
ensemble_cot_preference_model_fn = ensemble_optimizer.compile(ensemble_cot_preference_model)
eval_fn(ensemble_cot_preference_model_fn)

Average Metric: 352 / 500  (70.4): 100%|██████████| 500/500 [05:00<00:00,  1.67it/s]

Average Metric: 352 / 500  (70.4%)


70.4

# MIPRO (Multi-prompt Instruction Proposal Optimizer)

https://twitter.com/kristahopsalong/status/1766166198079889737

* Takes a teacher and a student LLMs, a dataset, and a metric.
* Multi stage optimization
    * The teacher looks at inputs and outputs and summarize the data
    * The student generates good traces for a few input where the metric is validated
    * The teacher create instruction given the good traces and the summary of the data
    * Use some kind of Bayes Optimization to search over instructions and examples


In [23]:
teacher = lm
student = lm

mipro = MIPRO(prompt_model=teacher, task_model=student, metric=em_metric, init_temperature=1.0, num_candidates=8)
kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=4)
mipro_preference_model = mipro.compile(student=PrefPredict(), trainset=train, num_trials=20, max_bootstrapped_demos=4, 
                                        max_labeled_demos=4, eval_kwargs=kwargs, requires_permission_to_run=False)



Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 1401 examples in dev set * 20 trials * # of LM calls in your program = (28020 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 8 * 1 lm calls in program = 18 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:



  1%|          | 10/1401 [00:00<00:03, 395.40it/s]


Bootstrapped 4 full traces after 11 examples in round 0.


  0%|          | 4/1401 [00:00<00:04, 290.78it/s]


Bootstrapped 4 full traces after 5 examples in round 0.


...

Bootstrapped 4 full traces after 6 examples in round 0.


  0%|          | 7/1401 [00:00<00:04, 344.58it/s]


Bootstrapped 4 full traces after 8 examples in round 0.


  1%|          | 8/1401 [00:00<00:03, 401.93it/s]

Bootstrapped 4 full traces after 9 examples in round 0.


In [ ]:
eval_fn(mipro_preference_model)

Average Metric: 333 / 500  (66.6): 100%|██████████| 500/500 [00:44<00:00, 11.24it/s]

Average Metric: 333 / 500  (66.6%)


66.6